In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
# Debugging: Plotting images for k=1 and k=2

# Ranking FFT frequencies based on absolute value (sqrt(a^2+b^2))

phi = np.load("Phi_201snap.npy")
df = pd.DataFrame()
i = 160
my_dpi = 192

# Define values for k
k_values = [1, 2]
data = phi[:,:,i]

plt.figure(figsize=(294/my_dpi, 165/my_dpi), dpi=my_dpi)
plt.imshow(data, cmap='gray')
plt.axis("off")
plt.savefig('2.4.png', bbox_inches='tight', pad_inches=0)
plt.close()

# Compute 1D (in r) and 2D FFT's
fft = np.fft.fft(data, axis=0)
fft2 = np.fft.fft2(data)

for k in k_values:
    
    # 1D
    # Sort fft by absolute value in descending order
    fft_sorted = np.sort(np.abs(fft.reshape(-1)))[::-1]
    
    # Take the top k frequencies from fft_sorted by defining threshold & mask smaller values
    threshold = fft_sorted[k]
    mask = np.abs(fft)>threshold
    masked = fft * mask

    # Reconstruct the data from masked using inverse FFT
    newdata = np.fft.ifft(masked).real

    # 2D
    # Sort fft by absolute value in descending order
    fft2_sorted = np.sort(np.abs(fft2.reshape(-1)))[::-1]

    # Take the top k frequencies from fft_sorted by defining threshold & mask smaller values
    threshold2 = fft2_sorted[k]
    mask2 = np.abs(fft2)>threshold2
    masked2 = fft2 * mask2
    
    # Reconstruct the data from masked2 using inverse FFT2
    newdata2 = np.fft.ifft2(masked2).real

    plt.figure(figsize=(294/my_dpi, 165/my_dpi), dpi=my_dpi)
    plt.imshow(newdata2, cmap='gray')
    plt.axis("off")
    plt.savefig('2.4new.png', bbox_inches='tight', pad_inches=0)
    plt.close()

In [11]:
# Ranking FFT frequencies based on absolute value (sqrt(a^2+b^2))

phi = np.load("Phi_201snap.npy")
df = pd.DataFrame()

# Define values for k
k_values = [1, 2, 5, 10, 20, 50, 75, 100]

for k in k_values:
    sum_of_squares1D, sum_of_squares2D = 0, 0
    max1D, min1D, max2D, min2D = -1000, 1000, -1000, 1000
    
    for i in range(201):
        data = phi[:,:,i]

        # Compute 1D (in r) and 2D FFT's
        fft = np.fft.fft(data, axis=0)
        fft2 = np.fft.fft2(data)

        # 1D
        # Sort fft by absolute value in descending order
        fft_sorted = np.sort(np.abs(fft.reshape(-1)))[::-1]
        
        # Take the top k frequencies from fft_sorted by defining threshold & mask smaller values
        threshold = fft_sorted[k]
        mask = np.abs(fft)>threshold
        masked = fft * mask

        # Reconstruct the data from masked using inverse FFT
        newdata = np.fft.ifft(masked).real
        
        # Add squared differences to total sum, update min/max values for later
        diff1D = data - newdata
        sum_of_squares1D += np.sum(diff1D**2)
        if np.max(diff1D) > max1D:
            max1D = np.max(diff1D)
        if np.min(diff1D) < min1D:
            min1D = np.min(diff1D)

        # 2D
        # Sort fft by absolute value in descending order
        fft2_sorted = np.sort(np.abs(fft2.reshape(-1)))[::-1]

        # Take the top k frequencies from fft_sorted by defining threshold & mask smaller values
        threshold2 = fft2_sorted[k]
        mask2 = np.abs(fft2)>threshold2
        masked2 = fft2 * mask2
        
        # Reconstruct the data from masked2 using inverse FFT2
        newdata2 = np.fft.ifft2(masked2).real
        
        # Add squared differences to total sum, update min/max values for later
        diff2D = data - newdata2
        sum_of_squares2D += np.sum(diff2D**2)
        if np.max(diff2D) > max2D:
            max2D = np.max(diff2D)
        if np.min(diff2D) < min2D:
            min2D = np.min(diff2D)

    # Compute RMSE's and CR's of 1D and 2D cases, add to dataframe
    range1D = max1D - min1D
    rms1D = np.sqrt(sum_of_squares1D / (101*227*201))
    rmse1D = rms1D / (100*range1D)
    cr1D = 101 / k
    cr_string1D = "CR1D_k=" + str(k)
    rmse_string1D = "RMSE1D_k=" + str(k)
    df.at[0, cr_string1D] = cr1D
    df.at[0, rmse_string1D] = rmse1D

    range2D = max2D - min2D
    rms2D = np.sqrt(sum_of_squares2D / (101*227*201))
    rmse2D = rms2D / (100*range2D)
    cr2D = (101*227) / k
    cr_string2D = "CR2D_k=" + str(k)
    rmse_string2D = "RMSE2D_k=" + str(k)
    df.at[0, cr_string2D] = cr2D
    df.at[0, rmse_string2D] = rmse2D
        
df.to_pickle("analysis2.4.pkl")
df

,CR1D_k=1,RMSE1D_k=1,CR2D_k=1,RMSE2D_k=1,CR1D_k=2,RMSE1D_k=2,CR2D_k=2,RMSE2D_k=2,CR1D_k=5,RMSE1D_k=5,...,CR2D_k=50,RMSE2D_k=50,CR1D_k=75,RMSE1D_k=75,CR2D_k=75,RMSE2D_k=75,CR1D_k=100,RMSE1D_k=100,CR2D_k=100,RMSE2D_k=100
0,101.0,0.000291,22927.0,0.000256,50.5,0.000291,11463.5,0.000256,20.2,0.00028,...,458.54,0.000142,1.346667,0.000041,305.693333,0.000158,1.01,0.000034,229.27,0.000145


In [10]:
# Ranking frequencies based on absolute value of real part

phi = np.load("Phi_201snap.npy")
df = pd.DataFrame()

# Define values for k
k_values = [1, 2, 5, 10, 20, 50, 75, 100]

for k in k_values:
    sum_of_squares1D, sum_of_squares2D = 0, 0
    max1D, min1D, max2D, min2D = -1000, 1000, -1000, 1000
    
    for i in range(201):
        data = phi[:,:,i]

        # Compute 1D (in r) and 2D FFT's
        fft = np.fft.fft(data, axis=0)
        fft2 = np.fft.fft2(data)

        # 1D
        # Sort fft by absolute value of real part in descending order
        fft_sorted = np.sort(np.abs(np.real(fft.reshape(-1))))[::-1]
        
        # Take the top k frequencies from fft_sorted by defining threshold & mask smaller values
        threshold = fft_sorted[k]
        mask = np.abs(np.real(fft))>threshold
        masked = fft * mask

        # Reconstruct the data from masked using inverse FFT
        newdata = np.fft.ifft(masked).real
        
        # Add squared differences to total sum, update min/max values for later
        diff1D = data - newdata
        sum_of_squares1D += np.sum(diff1D**2)
        if np.max(diff1D) > max1D:
            max1D = np.max(diff1D)
        if np.min(diff1D) < min1D:
            min1D = np.min(diff1D)

        # 2D
        # Sort fft by absolute value of real part in descending order
        fft2_sorted = np.sort(np.abs(np.real(fft2.reshape(-1))))[::-1]

        # Take the top k frequencies from fft_sorted by defining threshold & mask smaller values
        threshold2 = fft2_sorted[k]
        mask2 = np.abs(np.real(fft2))>threshold2
        masked2 = fft2 * mask2
        
        # Reconstruct the data from masked2 using inverse FFT2
        newdata2 = np.fft.ifft2(masked2).real
        
        # Add squared differences to total sum, update min/max values for later
        diff2D = data - newdata2
        sum_of_squares2D += np.sum(diff2D**2)
        if np.max(diff2D) > max2D:
            max2D = np.max(diff2D)
        if np.min(diff2D) < min2D:
            min2D = np.min(diff2D)

    # Compute RMSE's and CR's of 1D and 2D cases, add to dataframe
    range1D = max1D - min1D
    rms1D = np.sqrt(sum_of_squares1D / (101*227*201))
    rmse1D = rms1D / (100*range1D)
    cr1D = 101 / k
    cr_string1D = "CR1D_k=" + str(k)
    rmse_string1D = "RMSE1D_k=" + str(k)
    df.at[0, cr_string1D] = cr1D
    df.at[0, rmse_string1D] = rmse1D

    range2D = max2D - min2D
    rms2D = np.sqrt(sum_of_squares2D / (101*227*201))
    rmse2D = rms2D / (100*range2D)
    cr2D = (101*227) / k
    cr_string2D = "CR2D_k=" + str(k)
    rmse_string2D = "RMSE2D_k=" + str(k)
    df.at[0, cr_string2D] = cr2D
    df.at[0, rmse_string2D] = rmse2D
        
df.to_pickle("analysis2.4.pkl")
df

,CR1D_k=1,RMSE1D_k=1,CR2D_k=1,RMSE2D_k=1,CR1D_k=2,RMSE1D_k=2,CR2D_k=2,RMSE2D_k=2,CR1D_k=5,RMSE1D_k=5,...,CR2D_k=50,RMSE2D_k=50,CR1D_k=75,RMSE1D_k=75,CR2D_k=75,RMSE2D_k=75,CR1D_k=100,RMSE1D_k=100,CR2D_k=100,RMSE2D_k=100
0,101.0,0.000291,22927.0,0.000262,50.5,0.000291,11463.5,0.000265,20.2,0.00028,...,458.54,0.000288,1.346667,0.000041,305.693333,0.000285,1.01,0.000034,229.27,0.000252


In [9]:
# # Instead of looping through 201 images (in cell below), can do the 2D FFT on the whole phi array all at once. 
# # Easier then to do the 201*N RMSE thing.

# phi = np.load("Phi_201snap.npy")
# df = pd.DataFrame(index=np.arange(201))

# # Define values for k
# k_values = [1, 2, 5, 10, 20, 50, 75, 100]

# # Compute 1D (in r) and 2D (r, z) FFT's
# fft = np.fft.fft(phi, axis=0)
# fft2 = np.fft.fft2(phi, axes=(0, 1))

# for k in k_values:
#     # 1D
#     # Keep only the top k frequencies
#     fft_compressed1D = np.zeros_like(fft)
#     fft_compressed1D[:k] = fft[:k]
    
#     # Reconstruct the data using inverse FFT
#     reconstructed1D = np.fft.ifft(fft_compressed1D).real
    
#     # Compute the error (RMSE) and compression ratio
#     diff1D = data - reconstructed1D
#     rmse1D = np.sqrt(np.mean(diff1D**2))
#     cr1D = 101 / k
    
#     # Add to dataframe
#     # cr_string1D = "CR1D_k=" + str(k)
#     rmse_string1D = "RMSE1D_k=" + str(k)
#     # df.at[i, cr_string1D] = cr1D
#     df.at[i, rmse_string1D] = rmse1D
    
#     # 2D
#     # Keep only the top k frequencies
#     fft_compressed2D = np.zeros_like(fft2)
#     fft_compressed2D[:k, :k] = fft2[:k, :k]
    
#     # Reconstruct the data using inverse FFT
#     reconstructed2D = np.fft.ifft2(fft_compressed2D).real
    
#     # Compute the error (RMSE) and compression ratio
#     diff2D = data - reconstructed2D
#     rmse2D = np.sqrt(np.mean(diff2D**2))
#     cr2D = (101 * 227) / (k * k)
    
#     # Add to dataframe
#     # cr_string2D = "CR2D_k=" + str(k)
#     rmse_string2D = "RMSE2D_k=" + str(k)
#     # df.at[i, cr_string2D] = cr2D
#     df.at[i, rmse_string2D] = rmse2D
        
# # df.to_pickle("analysis2.4.pkl")
# df
np.abs(np.real(-33+4j))

33.0